In [1]:
!python -m pip install pennylane numpy scipy pandas plotly tqdm nbformat



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 kB 802.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 934.3/934.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 34.9 MB/s eta 0:00:00


In [10]:
import pennylane as qml
import numpy as np
import plotly
import plotly.graph_objects as go
from fractions import Fraction

print("PennyLane version:", qml.__version__)
print("NumPy version:", np.__version__)
print("Plotly version:", plotly.__version__)



PennyLane version: 0.43.2
NumPy version: 2.0.2
Plotly version: 5.24.1


In [3]:
import numpy as np



Pz0 = np.array([[1, 0],
                [0, 0]])

Pz1 = np.array([[0, 0],
                [0, 1]])

Px0 = 0.5 * np.array([[1,  1],
                      [1,  1]])

Px1 = 0.5 * np.array([[1, -1],
                      [-1, 1]])

Py0 = 0.5 * np.array([[1, -1j],
                      [1j, 1]])

Py1 = 0.5 * np.array([[1,  1j],
                      [-1j, 1]])

pauli_measurements = {
    "Z": [Pz0, Pz1],
    "X": [Px0, Px1],
    "Y": [Py0, Py1]
}

print("Pauli measurement operators created.")


Pauli measurement operators created.


In [4]:
for basis, ops in pauli_measurements.items():
    total = ops[0] + ops[1]
    print(f"{basis} completeness check:")
    print(np.allclose(total, np.eye(2)))


Z completeness check:
True
X completeness check:
True
Y completeness check:
True


In [5]:
ket0 = np.array([[1],
                 [0]])
rho0 = ket0 @ ket0.T.conj()

for basis, ops in pauli_measurements.items():
    probs = [np.real(np.trace(P @ rho0)) for P in ops]
    print(f"{basis} measurement probabilities:", probs, "sum =", sum(probs))


Z measurement probabilities: [np.int64(1), np.int64(0)] sum = 1
X measurement probabilities: [np.float64(0.5), np.float64(0.5)] sum = 1.0
Y measurement probabilities: [np.float64(0.5), np.float64(0.5)] sum = 1.0


In [7]:
import numpy as np

states = {
    "|0>": np.array([[1], [0]], dtype=complex),
    "|1>": np.array([[0], [1]], dtype=complex),
    "|+>": np.array([[1], [1]], dtype=complex) / np.sqrt(2),
    "|->": np.array([[1], [-1]], dtype=complex) / np.sqrt(2),
    "|+i>": np.array([[1], [1j]], dtype=complex) / np.sqrt(2),
}

print("Reference states created:", list(states.keys()))


Reference states created: ['|0>', '|1>', '|+>', '|->', '|+i>']


In [8]:
density_matrices = {}

for name, ket in states.items():
    rho = ket @ ket.conj().T
    density_matrices[name] = rho
    print(f"{name} density matrix:\n", rho, "\n")


|0> density matrix:
 [[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]] 

|1> density matrix:
 [[0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]] 

|+> density matrix:
 [[0.5+0.j 0.5+0.j]
 [0.5+0.j 0.5+0.j]] 

|-> density matrix:
 [[ 0.5+0.j -0.5+0.j]
 [-0.5+0.j  0.5+0.j]] 

|+i> density matrix:
 [[0.5+0.j  0. -0.5j]
 [0. +0.5j 0.5+0.j ]] 



In [12]:
import numpy as np
import plotly.graph_objects as go
from fractions import Fraction

_CUBE_FACES = (
    (0, 1, 2), (0, 2, 3),
    (4, 5, 6), (4, 6, 7),
    (0, 1, 5), (0, 5, 4),
    (1, 2, 6), (1, 6, 5),
    (2, 3, 7), (2, 7, 6),
    (3, 0, 4), (3, 4, 7)
)

def _phase_to_pi_string(angle_rad: float) -> str:
    if np.isclose(angle_rad, 0.0):
        return "0"
    multiple = angle_rad / np.pi
    frac = Fraction(multiple).limit_denominator(16)
    n, d = frac.numerator, frac.denominator
    sign = "-" if n < 0 else ""
    n = abs(n)
    if d == 1:
        mag = f"{n}" if n != 1 else ""
    else:
        mag = f"{n}/{d}"
    return f"{sign}{mag}π" if mag else f"{sign}π"

def plot_density_matrix_histogram(rho, basis_labels=None, title="Density matrix"):
    rho = np.asarray(rho)
    dim = rho.shape[0]
    mags = np.abs(rho)
    phases = np.angle(rho)

    if basis_labels is None:
        basis_labels = [str(i) for i in range(dim)]

    meshes = []
    colorbar_added = False

    for i in range(dim):
        for j in range(dim):
            h = mags[i, j]
            phase = phases[i, j]

            x0, x1 = i - 0.45, i + 0.45
            y0, y1 = j - 0.45, j + 0.45

            vertices = (
                (x0, y0, 0), (x1, y0, 0), (x1, y1, 0), (x0, y1, 0),
                (x0, y0, h), (x1, y0, h), (x1, y1, h), (x0, y1, h)
            )

            x, y, z = zip(*vertices)
            ii, jj, kk = zip(*_CUBE_FACES)

            mesh = go.Mesh3d(
                x=x, y=y, z=z,
                i=ii, j=jj, k=kk,
                intensity=[phase] * len(vertices),
                colorscale="HSV",
                cmin=-np.pi,
                cmax=np.pi,
                showscale=not colorbar_added,
                colorbar=dict(
                    title="phase",
                    tickvals=[-np.pi, -np.pi/2, 0, np.pi/2, np.pi],
                    ticktext=["-π", "-π/2", "0", "π/2", "π"]
                ) if not colorbar_added else None
            )

            meshes.append(mesh)
            colorbar_added = True

    fig = go.Figure(data=meshes)
    fig.update_layout(
        title=title,
        scene=dict(
            xaxis=dict(title="i", tickvals=list(range(dim)), ticktext=basis_labels),
            yaxis=dict(title="j", tickvals=list(range(dim)), ticktext=basis_labels),
            zaxis=dict(title="|ρᵢⱼ|")
        )
    )
    fig.show()



In [13]:
plot_density_matrix_histogram(
    density_matrices["|+>"],
    basis_labels=["0", "1"],
    title="Density matrix of |+> state"
)


## Reference State Preparation

The following single-qubit reference states were used for tomography.

| State | Circuit Description |
|------|---------------------|
| |0⟩ | Initialized in the computational basis |
| |1⟩ | X gate applied to |0⟩ |
| |+⟩ | Hadamard gate applied to |0⟩ |
| |−⟩ | X gate followed by Hadamard gate |
| |+i⟩ | Hadamard gate followed by S (phase) gate |


In [14]:
SHOTS = 2000
np.random.seed(42)


In [15]:
def sample_measurement_counts(rho, projectors, shots):
    probs = [np.real(np.trace(P @ rho)) for P in projectors]
    probs = np.maximum(probs, 0.0)
    probs = probs / np.sum(probs)
    counts = np.random.multinomial(shots, probs)
    return counts, probs


In [16]:
rho_plus = density_matrices["|+>"]

measurement_results = {}

for basis, ops in pauli_measurements.items():
    counts, probs = sample_measurement_counts(rho_plus, ops, SHOTS)
    measurement_results[basis] = {
        "counts": counts,
        "probs": probs
    }
    print(f"{basis} basis counts:", counts)
    print(f"{basis} basis probs :", probs)
    print()


Z basis counts: [ 980 1020]
Z basis probs : [0.5 0.5]

X basis counts: [2000    0]
X basis probs : [1. 0.]

Y basis counts: [1031  969]
Y basis probs : [0.5 0.5]



In [18]:
def expectation_from_counts(counts):
    return (counts[0] - counts[1]) / np.sum(counts)


In [19]:
expvals = {}

for basis in ["X", "Y", "Z"]:
    counts = measurement_results[basis]["counts"]
    expvals[basis] = expectation_from_counts(counts)

print("Expectation values:")
for k, v in expvals.items():
    print(f"<{k}> = {v:.3f}")


Expectation values:
<X> = 1.000
<Y> = 0.031
<Z> = -0.020


In [20]:
I = np.eye(2)
X = np.array([[0, 1],
              [1, 0]])
Y = np.array([[0, -1j],
              [1j, 0]])
Z = np.array([[1, 0],
              [0, -1]])


In [21]:
rho_reconstructed = 0.5 * (
    I
    + expvals["X"] * X
    + expvals["Y"] * Y
    + expvals["Z"] * Z
)

print("Reconstructed density matrix:")
print(rho_reconstructed)


Reconstructed density matrix:
[[0.49+0.j     0.5 -0.0155j]
 [0.5 +0.0155j 0.51+0.j    ]]


In [22]:
plot_density_matrix_histogram(
    rho_reconstructed,
    basis_labels=["0", "1"],
    title="Reconstructed density matrix (|+⟩)"
)


In [23]:
plot_density_matrix_histogram(
    density_matrices["|+>"],
    basis_labels=["0", "1"],
    title="True density matrix (|+⟩)"
)


In [25]:
def fidelity_pure(rho_true, rho_est):
    return np.real(np.trace(rho_true @ rho_est))


In [26]:
from numpy.linalg import svd

def trace_distance(rho_true, rho_est):
    svals = svd(rho_true - rho_est, compute_uv=False)
    return 0.5 * np.sum(svals)


In [27]:
rho_true = density_matrices["|+>"]

F = fidelity_pure(rho_true, rho_reconstructed)
D = trace_distance(rho_true, rho_reconstructed)

print(f"Fidelity        : {F:.4f}")
print(f"Trace distance  : {D:.4f}")


Fidelity        : 1.0000
Trace distance  : 0.0184


## Validation Results

For the |+⟩ state reconstructed using Pauli projective measurements:

- Fidelity was close to unity, indicating accurate reconstruction.
- Trace distance remained small, with deviations dominated by shot noise.
- Increasing the number of shots improves reconstruction quality.

Shot noise and finite sampling are the primary error sources in this experiment.
